In [1]:
cd ..

C:\Users\rajag\Desktop\convolve\convolve_epoch_1


In [2]:
cd data

C:\Users\rajag\Desktop\convolve\convolve_epoch_1\data


In [3]:
# import libraries
import pandas as pd 
import numpy as np
import json

In [4]:
# custom settings 
pd.set_option('display.max_colwidth', None)

In [5]:
# get the data 
with open('train.json', 'r') as f:
    dict_train = json.load(f)

df = pd.DataFrame.from_dict(dict_train, orient='index')
df.reset_index(level=0, inplace=True)
df.rename(columns = {'index':'log',0:'status'}, inplace = True)

In [69]:
# Prepare the training data
df_parsed = df.copy(deep=True)
df_parsed["log"]= df_parsed["log"].str.split(":", n = 3, expand = False)
df_temp = df_parsed.copy(deep=True)
df_temp = pd.DataFrame(df_parsed['log'].to_list(), columns=['log1','log2','log3','log4'])
df_temp['status'] = df_parsed['status']
df4 = df_temp[df_temp['log4'].isnull()].copy(deep=True)
df5 = df_temp[df_temp['log4'].notnull()].copy(deep=True)
train1 = df4.drop(['log4', 'log1', 'log2'], axis=1)
train2 = df5.drop(['log3', 'log1', 'log2'], axis=1)
train1.rename(columns = {'log3':'log'}, inplace = True)
train2.rename(columns = {'log4':'log'}, inplace = True)
tmp_list = [train1,train2]
train = pd.concat(tmp_list)
train = train.dropna()

# Method 1

In [7]:
# https://www.analyticsvidhya.com/blog/2022/02/sentiment-analysis-using-transformers/

In [8]:
train1 = train.copy(deep=True)

In [9]:
train1["status"] = train1["status"].map({"abnormal": 0, "normal": 1})

In [10]:
# method 5, custom sampler
total_normal = train1[train1["status"] == 1]
total_abnormal = train1[train1["status"] == 0]

sampled_total_normal = total_normal.sample(n=69692)

normal_distributed_df = pd.concat([total_abnormal, sampled_total_normal])

balanced_train2 = normal_distributed_df.sample(frac=1,random_state=42)

In [11]:
balanced_train2.head()

log  \
2606855                     J14-U01 RAS KERNEL INFO generating core.16136\n   
2445412  J12-U11 RAS KERNEL INFO 1005 floating point alignment exceptions\n   
4150103                   J16-U01 RAS KERNEL FATAL data TLB error interrupt   
4087324                   J16-U01 RAS KERNEL FATAL data TLB error interrupt   
4140189                   J16-U01 RAS KERNEL FATAL data TLB error interrupt   

         status  
2606855       1  
2445412       1  
4150103       0  
4087324       0  
4140189       0

In [ ]:
pip install transformers

# Method 2 


In [ ]:
# https://www.kaggle.com/code/maroberti/fastai-with-transformers-bert-roberta

In [51]:
balanced_train2.head()

log  \
2606855                     J14-U01 RAS KERNEL INFO generating core.16136\n   
2445412  J12-U11 RAS KERNEL INFO 1005 floating point alignment exceptions\n   
4150103                   J16-U01 RAS KERNEL FATAL data TLB error interrupt   
4087324                   J16-U01 RAS KERNEL FATAL data TLB error interrupt   
4140189                   J16-U01 RAS KERNEL FATAL data TLB error interrupt   

         status  
2606855       1  
2445412       1  
4150103       0  
4087324       0  
4140189       0

In [ ]:
pip install -q transformers
pip install torchvision 

In [ ]:
pip install fastai==1.0.58 --user

In [ ]:
pip install transformers==2.5.1 --user

In [12]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path 

import os

import torch
import torch.optim as optim

import random 

# fastai
from fastai import *
from fastai.text import *
from fastai.callback import *

# transformers
from transformers import PreTrainedModel, PreTrainedTokenizer, PretrainedConfig

from transformers import BertForSequenceClassification, BertTokenizer, BertConfig
from transformers import RobertaForSequenceClassification, RobertaTokenizer, RobertaConfig
from transformers import XLNetForSequenceClassification, XLNetTokenizer, XLNetConfig
from transformers import XLMForSequenceClassification, XLMTokenizer, XLMConfig
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, DistilBertConfig

In [13]:
import fastai
import transformers
print('fastai version :', fastai.__version__)
print('transformers version :', transformers.__version__)

fastai version : 1.0.58
transformers version : 2.5.1


In [14]:
MODEL_CLASSES = {
    'bert': (BertForSequenceClassification, BertTokenizer, BertConfig),
    'xlnet': (XLNetForSequenceClassification, XLNetTokenizer, XLNetConfig),
    'xlm': (XLMForSequenceClassification, XLMTokenizer, XLMConfig),
    'roberta': (RobertaForSequenceClassification, RobertaTokenizer, RobertaConfig),
    'distilbert': (DistilBertForSequenceClassification, DistilBertTokenizer, DistilBertConfig)
}

In [15]:
# Parameters
seed = 42
use_fp16 = False
bs = 16

model_type = 'roberta'
pretrained_model_name = 'roberta-base'

# model_type = 'bert'
# pretrained_model_name='bert-base-uncased'

# model_type = 'distilbert'
# pretrained_model_name = 'distilbert-base-uncased'

#model_type = 'xlm'
#pretrained_model_name = 'xlm-clm-enfr-1024'

# model_type = 'xlnet'
# pretrained_model_name = 'xlnet-base-cased'

In [16]:
model_class, tokenizer_class, config_class = MODEL_CLASSES[model_type]

In [17]:
model_class.pretrained_model_archive_map.keys()

dict_keys(['roberta-base', 'roberta-large', 'roberta-large-mnli', 'distilroberta-base', 'roberta-base-openai-detector', 'roberta-large-openai-detector'])

In [18]:

# Function to set the seed for generating random numbers.

def seed_all(seed_value):
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False

In [19]:
seed_all(seed)

In [20]:
class TransformersBaseTokenizer():
    """Wrapper around PreTrainedTokenizer to be compatible with fast.ai"""
    def __init__(self, pretrained_tokenizer: PreTrainedTokenizer, model_type = 'bert', **kwargs):
        self._pretrained_tokenizer = pretrained_tokenizer
        self.max_seq_len = pretrained_tokenizer.max_len
        self.model_type = model_type

    def __call__(self, *args, **kwargs): 
        return self

    def tokenizer(self, t:str):
        """Limits the maximum sequence length and add the spesial tokens"""
        CLS = self._pretrained_tokenizer.cls_token
        SEP = self._pretrained_tokenizer.sep_token
        if self.model_type in ['roberta']:
            tokens = self._pretrained_tokenizer.tokenize(t, add_prefix_space=True)[:self.max_seq_len - 2]
            tokens = [CLS] + tokens + [SEP]
        else:
            tokens = self._pretrained_tokenizer.tokenize(t)[:self.max_seq_len - 2]
            if self.model_type in ['xlnet']:
                tokens = tokens + [SEP] +  [CLS]
            else:
                tokens = [CLS] + tokens + [SEP]
        return tokens

In [21]:
transformer_tokenizer = tokenizer_class.from_pretrained(pretrained_model_name)
transformer_base_tokenizer = TransformersBaseTokenizer(pretrained_tokenizer = transformer_tokenizer, model_type = model_type)
fastai_tokenizer = Tokenizer(tok_func = transformer_base_tokenizer, pre_rules=[], post_rules=[])

In [22]:
class TransformersVocab(Vocab):
    def __init__(self, tokenizer: PreTrainedTokenizer):
        super(TransformersVocab, self).__init__(itos = [])
        self.tokenizer = tokenizer
    
    def numericalize(self, t:Collection[str]) -> List[int]:
        "Convert a list of tokens `t` to their ids."
        return self.tokenizer.convert_tokens_to_ids(t)
        #return self.tokenizer.encode(t)

    def textify(self, nums:Collection[int], sep=' ') -> List[str]:
        "Convert a list of `nums` to their tokens."
        nums = np.array(nums).tolist()
        return sep.join(self.tokenizer.convert_ids_to_tokens(nums)) if sep is not None else self.tokenizer.convert_ids_to_tokens(nums)
    
    def __getstate__(self):
        return {'itos':self.itos, 'tokenizer':self.tokenizer}

    def __setstate__(self, state:dict):
        self.itos = state['itos']
        self.tokenizer = state['tokenizer']
        self.stoi = collections.defaultdict(int,{v:k for k,v in enumerate(self.itos)})

In [23]:
transformer_vocab =  TransformersVocab(tokenizer = transformer_tokenizer)
numericalize_processor = NumericalizeProcessor(vocab=transformer_vocab)

tokenize_processor = TokenizeProcessor(tokenizer=fastai_tokenizer, include_bos=False, include_eos=False)

transformer_processor = [tokenize_processor, numericalize_processor]

In [24]:
pad_first = bool(model_type in ['xlnet'])
pad_idx = transformer_tokenizer.pad_token_id

In [25]:
# a check i believe
tokens = transformer_tokenizer.tokenize('Salut c est moi, Hello it s me')
print(tokens)
ids = transformer_tokenizer.convert_tokens_to_ids(tokens)
print(ids)
transformer_tokenizer.convert_ids_to_tokens(ids)

['Sal', 'ut', 'Ġc', 'Ġest', 'Ġmo', 'i', ',', 'ĠHello', 'Ġit', 'Ġs', 'Ġme']
[18111, 1182, 740, 3304, 7458, 118, 6, 20920, 24, 579, 162]


['Sal', 'ut', 'Ġc', 'Ġest', 'Ġmo', 'i', ',', 'ĠHello', 'Ġit', 'Ġs', 'Ġme']

In [87]:
test = pd.read_csv("test.csv")

In [46]:
df_parsed = test.copy(deep=True)
df_parsed[" Log"]= df_parsed[" Log"].str.split(":", n = 3, expand = False)
df_temp = df_parsed.copy(deep=True)
df_temp = pd.DataFrame(df_parsed[' Log'].to_list(), columns=['log1','log2','log3','log4'])
# df_temp['status'] = df_parsed['status']
df4 = df_temp[df_temp['log4'].isnull()].copy(deep=True)
df5 = df_temp[df_temp['log4'].notnull()].copy(deep=True)
train1 = df4.drop(['log4', 'log1', 'log2'], axis=1)
train2 = df5.drop(['log3', 'log1', 'log2'], axis=1)
train1.rename(columns = {'log3':'log'}, inplace = True)
train2.rename(columns = {'log4':'log'}, inplace = True)
tmp_list = [train1,train2]
train = pd.concat(tmp_list)
test = train.dropna()

In [47]:
test.head()

log
1            J16-U01 RAS KERNEL FATAL data TLB error interrupt\n
2            J16-U01 RAS KERNEL FATAL data TLB error interrupt\n
3                 J03-U11 RAS KERNEL INFO generating core.6463\n
4            J16-U01 RAS KERNEL FATAL data TLB error interrupt\n
5  J08-U01 RAS KERNEL INFO CE sym 35, at 0x06a7bd60, mask 0x40\n

In [49]:
databunch = (TextList.from_df(balanced_train2, cols='log', processor=transformer_processor)
             .split_by_rand_pct(0.1,seed=seed)
             .label_from_df(cols= 'status')
             .add_test(test)
             .databunch(bs=bs, pad_first=pad_first, pad_idx=pad_idx))

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [ ]:
print('[CLS] token :', transformer_tokenizer.cls_token)
print('[SEP] token :', transformer_tokenizer.sep_token)
print('[PAD] token :', transformer_tokenizer.pad_token)
databunch.show_batch()

In [ ]:
print('[CLS] id :', transformer_tokenizer.cls_token_id)
print('[SEP] id :', transformer_tokenizer.sep_token_id)
print('[PAD] id :', pad_idx)
test_one_batch = databunch.one_batch()[0]
print('Batch shape : ',test_one_batch.shape)
print(test_one_batch)

In [50]:
# more to do 

# Method 3

In [ ]:
# https://towardsdatascience.com/approaches-to-sentimental-analysis-on-a-small-imbalanced-dataset-without-deep-learning-a314817e687

In [105]:
test = pd.read_csv("test.csv")

In [106]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 595300 entries, 0 to 595299
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   ID      595300 non-null  int64 
 1    Log    595300 non-null  object
dtypes: int64(1), object(1)
memory usage: 9.1+ MB


In [107]:
test.head(10)

ID  \
0   0   
1   1   
2   2   
3   3   
4   4   
5   5   
6   6   
7   7   
8   8   
9   9   

                                                                                                                                                        Log  
0        1124336301 2005.08.17 R13-M1-N8-C:J12-U01 2005-08-17-20.38.21.466368 R13-M1-N8-C:J12-U01 RAS KERNEL FATAL rts: kernel terminated for reason 1001\n  
1                      1118553175 2005.06.11 R30-M0-N9-C:J16-U01 2005-06-11-22.12.55.707149 R30-M0-N9-C:J16-U01 RAS KERNEL FATAL data TLB error interrupt\n  
2                      1118536033 2005.06.11 R30-M0-N9-C:J16-U01 2005-06-11-17.27.13.042387 R30-M0-N9-C:J16-U01 RAS KERNEL FATAL data TLB error interrupt\n  
3                           1117992566 2005.06.05 R30-M1-N6-C:J03-U11 2005-06-05-10.29.26.943462 R30-M1-N6-C:J03-U11 RAS KERNEL INFO generating core.6463\n  
4                      1118538965 2005.06.11 R30-M0-N9-C:J16-U01 2005-06-11-18.16.05.049256 R30-M0-N9-C:J16-U01 RAS KERNEL FATAL data TLB error interrupt\n  
5            1118074540 2005.06.06 R15-M0-N0-C:J08-U01 2005-06-06-09.15.40.112852 R15-M0-N0-C:J08-U01 RAS KERNEL INFO CE sym 35, at 0x06a7bd60, mask 0x40\n  
6                      1118541623 2005.06.11 R30-M0-N9-C:J16-U01 2005-06-11-19.00.23.903646 R30-M0-N9-C:J16-U01 RAS KERNEL FATAL data TLB error interrupt\n  
7                           1118149822 2005.06.07 R21-M0-N4-C:J17-U11 2005-06-07-06.10.22.237271 R21-M0-N4-C:J17-U11 RAS KERNEL INFO generating core.2364\n  
8                      1118553027 2005.06.11 R30-M0-N9-C:J16-U01 2005-06-11-22.10.27.830555 R30-M0-N9-C:J16-U01 RAS KERNEL FATAL data TLB error interrupt\n  
9   1118171330 2005.06.07 R23-M0-N0-I:J18-U01 2005-06-07-12.08.50.515997 R23-M0-N0-I:J18-U01 RAS KERNEL INFO ciod: Message code 0 is not 51 or 4294967295\n

In [108]:
df_parsed = test.copy(deep=True)
df_parsed[" Log"]= df_parsed[" Log"].str.split(":", n = 3, expand = False)
df_temp = df_parsed.copy(deep=True)
df_temp = pd.DataFrame(df_parsed[' Log'].to_list(), columns=['log1','log2','log3','log4'])
df_temp['ID'] = df_parsed['ID']
df4 = df_temp[df_temp['log4'].isnull()].copy(deep=True)
df5 = df_temp[df_temp['log4'].notnull()].copy(deep=True)
train1 = df4.drop(['log4', 'log1', 'log2'], axis=1)
train2 = df5.drop(['log3', 'log1', 'log2'], axis=1)
train1.rename(columns = {'log3':'log'}, inplace = True)
train2.rename(columns = {'log4':'log'}, inplace = True)
tmp_list = [train1,train2]
train = pd.concat(tmp_list)
test = train
# test = train.dropna()

In [112]:
test = test.fillna(test.mode().iloc[0])

In [113]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 595300 entries, 1 to 595295
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   log     595300 non-null  object
 1   ID      595300 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 13.6+ MB


In [119]:
test.head(10)

log  ID
1                  J16-U01 RAS KERNEL FATAL data TLB error interrupt\n   1
2                  J16-U01 RAS KERNEL FATAL data TLB error interrupt\n   2
3                       J03-U11 RAS KERNEL INFO generating core.6463\n   3
4                  J16-U01 RAS KERNEL FATAL data TLB error interrupt\n   4
5        J08-U01 RAS KERNEL INFO CE sym 35, at 0x06a7bd60, mask 0x40\n   5
6                  J16-U01 RAS KERNEL FATAL data TLB error interrupt\n   6
7                       J17-U11 RAS KERNEL INFO generating core.2364\n   7
8                  J16-U01 RAS KERNEL FATAL data TLB error interrupt\n   8
10                       J14-U01 RAS KERNEL INFO generating core.228\n  10
11  J12-U11 RAS KERNEL INFO instruction cache parity error corrected\n  11

In [122]:
test = test.sort_values("ID")

In [123]:
test.head(10)

log  ID
0                            kernel terminated for reason 1001\n   0
1            J16-U01 RAS KERNEL FATAL data TLB error interrupt\n   1
2            J16-U01 RAS KERNEL FATAL data TLB error interrupt\n   2
3                 J03-U11 RAS KERNEL INFO generating core.6463\n   3
4            J16-U01 RAS KERNEL FATAL data TLB error interrupt\n   4
5  J08-U01 RAS KERNEL INFO CE sym 35, at 0x06a7bd60, mask 0x40\n   5
6            J16-U01 RAS KERNEL FATAL data TLB error interrupt\n   6
7                 J17-U11 RAS KERNEL INFO generating core.2364\n   7
8            J16-U01 RAS KERNEL FATAL data TLB error interrupt\n   8
9                       Message code 0 is not 51 or 4294967295\n   9

In [124]:
train = balanced_train2.copy(deep=True)

In [125]:
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [126]:
tokenizer = TweetTokenizer()
vectorizer = TfidfVectorizer(ngram_range=(1, 3), tokenizer=tokenizer.tokenize)
full_text = list(train['log'].values) + list(test['log'].values)
vectorizer.fit(full_text)
train_vectorized = vectorizer.transform(train['log'])
test_vectorized = vectorizer.transform(test['log'])
y = train['status']

C:\Users\rajag\AppData\Roaming\Python\Python38\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [127]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

logreg = LogisticRegression(class_weight='balanced')
ovr = OneVsRestClassifier(logreg).fit(train_vectorized,y)

In [128]:
len(train)

139384

In [129]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(ovr, train_vectorized, y, scoring='f1_macro', n_jobs=-1, cv=3)
print('Cross-validation mean f1_score {0:.2f}%, std {1:.2f}.'.format(np.mean(scores), np.std(scores)))

Cross-validation mean f1_score 1.00%, std 0.00.


In [130]:
test_vectorized

<595300x198716 sparse matrix of type '<class 'numpy.float64'>'
	with 14625118 stored elements in Compressed Sparse Row format>

In [131]:
pred = ovr.predict_proba(test_vectorized)


In [132]:
# more shit in the article to make it better, but first..... check this on the original dataset

In [133]:
len(test)

595300

In [134]:
len(pred.argmax(1))

595300

In [144]:
count = 0
for i in pred.argmax(1):
    if i == 0:
        count += 1
        


In [147]:
164876/595300

0.27696287586091045

In [146]:
print(count)

164876


In [137]:
sub = test

In [138]:
sub.head()

log  ID
0                  kernel terminated for reason 1001\n   0
1  J16-U01 RAS KERNEL FATAL data TLB error interrupt\n   1
2  J16-U01 RAS KERNEL FATAL data TLB error interrupt\n   2
3       J03-U11 RAS KERNEL INFO generating core.6463\n   3
4  J16-U01 RAS KERNEL FATAL data TLB error interrupt\n   4

In [139]:
sub.head()

log  ID
0                  kernel terminated for reason 1001\n   0
1  J16-U01 RAS KERNEL FATAL data TLB error interrupt\n   1
2  J16-U01 RAS KERNEL FATAL data TLB error interrupt\n   2
3       J03-U11 RAS KERNEL INFO generating core.6463\n   3
4  J16-U01 RAS KERNEL FATAL data TLB error interrupt\n   4

In [140]:
len(sub)

595300

In [142]:
sub['status'] = pred.argmax(1)
# sub.to_csv('sub.csv', index=False)

In [149]:
sub.head(20)

log  \
0                                                           kernel terminated for reason 1001\n   
1                                           J16-U01 RAS KERNEL FATAL data TLB error interrupt\n   
2                                           J16-U01 RAS KERNEL FATAL data TLB error interrupt\n   
3                                                J03-U11 RAS KERNEL INFO generating core.6463\n   
4                                           J16-U01 RAS KERNEL FATAL data TLB error interrupt\n   
5                                 J08-U01 RAS KERNEL INFO CE sym 35, at 0x06a7bd60, mask 0x40\n   
6                                           J16-U01 RAS KERNEL FATAL data TLB error interrupt\n   
7                                                J17-U11 RAS KERNEL INFO generating core.2364\n   
8                                           J16-U01 RAS KERNEL FATAL data TLB error interrupt\n   
9                                                      Message code 0 is not 51 or 4294967295\n   
10                                                J14-U01 RAS KERNEL INFO generating core.228\n   
11                           J12-U11 RAS KERNEL INFO instruction cache parity error corrected\n   
12                                          J16-U01 RAS KERNEL FATAL data TLB error interrupt\n   
13                                               J05-U01 RAS KERNEL INFO generating core.4279\n   
14                                          J16-U01 RAS KERNEL FATAL data TLB error interrupt\n   
15                                               J15-U01 RAS KERNEL INFO generating core.7572\n   
16                                          J16-U01 RAS KERNEL FATAL data TLB error interrupt\n   
17                                          J16-U01 RAS KERNEL FATAL data TLB error interrupt\n   
18   failed to read message prefix on control stream (CioStream socket to 172.16.96.116:33240\n   
19                                          J16-U01 RAS KERNEL FATAL data TLB error interrupt\n   

    ID  status  
0    0       1  
1    1       0  
2    2       0  
3    3       1  
4    4       0  
5    5       1  
6    6       0  
7    7       1  
8    8       0  
9    9       1  
10  10       1  
11  11       1  
12  12       0  
13  13       1  
14  14       0  
15  15       1  
16  16       0  
17  17       0  
18  18       0  
19  19       0

In [154]:
# sub = sub.drop(['log'], axis=1)
sub.rename(columns = {'status':'Label',}, inplace = True)
sub["Label"] = sub["Label"].map({0:"abnormal",1:"normal"})

In [156]:
sub.head(20)

ID     Label
0    0    normal
1    1  abnormal
2    2  abnormal
3    3    normal
4    4  abnormal
5    5    normal
6    6  abnormal
7    7    normal
8    8  abnormal
9    9    normal
10  10    normal
11  11    normal
12  12  abnormal
13  13    normal
14  14  abnormal
15  15    normal
16  16  abnormal
17  17  abnormal
18  18  abnormal
19  19  abnormal

In [ ]:
# so my drop na is droping about 1000 rows, so un sabka i can set 1 by default. Rest ka i have to map  the id and log

In [157]:
len(sub)

595300

In [158]:
sub.to_csv(r'C:\Users\rajag\Desktop\convolve\convolve_epoch_1\submission\submission.csv', index=False)